# Analiza danych – ćwiczenia 3.
### Pierwsza sieć neuronowa z Keras


##### Autor: mgr inż. Bartosz Czech

---

#### Wprowadzenie
Zanim zaczniemy budować sieć neuronową opartą na dane dotyczące polimorfizmów pojedynczego nukleotydu, zapoznamy się z budową modelu sieci neuronowej przy pomocy biblioteki Keras. Jako, że temat ćwiczeń dotyczy zbudowania modelu klasyfikacyjnego, przykład zamieszczony w niniejszym ćwiczeniu również ilustruje implementację modelu dla problemu klasyfikacyjnego. Zbiór danych pochodzi z Breast Cancer Wisconsin (Diagnostic). Zawiera 569 wierszy (reprezentujących pacjentów) oraz 30 kolumn numerycznych (cech). Celem ćwiczenia jest zbudowanie klasyfikatora umożliwiającego diagnostykę pacjentów. Domyślnie załadowany zbiór danych to obiekt typu scikit-learn Bunch object, który swoją strukturą przypomina słownik, jednak metody z biblioteki **pandas** nie mogą być na tym obiekcie zastosowane. Chcemy sklasyfikować typ raka (złośliwy/łagodny) wykorzystując następujące kodowanie 1 – malignant (złośliwy) lub 0 – benign (łagodny).

---

In [103]:
from keras.datasets import boston_housing
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras import optimizers
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.preprocessing import normalize
from sklearn.metrics import confusion_matrix

In [83]:
cancer = load_breast_cancer()

In [84]:
print(cancer.keys())

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


In [85]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [86]:
X_data = cancer.data # przypisanie tensoru
y_data = cancer.target # przypisanie wektoru z informacją o typie raka

Tworzymy zbiór traningowy oraz zbiór testowy. W tym modelu 30% losowych obserwacji zostało przypisanych do testowego zbioru testowego. 70% zostanie użytych do trenowania modelu. Zauważ, że argument stratify nie został uwzględniony – chcemy, aby dane były wylosowane bez uwzględnienia rozkładu zmiennej y.

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.35, random_state = 123) 

Tworzymy pusty model typu Sequential. Typ Sequential charakteryzuje się liniowym stosem warstw i jest najpopularniejszą architekturą sieci.

In [88]:
model = Sequential()

Tworzymy dwie ukryte warstwy sieci, z 10 neuronami w każdej warstwie. W każdej warstwie używamy sigmoidalnej funkcji aktywacji. Więcej informacji o funkcjach aktywacji w keras znajduje się [tutaj.](https://keras.io/activations/). 

### Funkcja liniowa
$f(x) = x$ 
* sygnał wejściowy = sygnał wejściowy
* niemożliwa optymalizacja (propagacja wsteczna)
* w pewnym sensie lepsza niż funkcja krokowa (wiele wyjść)
* wszystkie warstwy sieci neuronowej są funkcją liniową; ostateczne wyjście jest kombinacją funkcji liniowych
* ANN z funkcją identyczności = model regresji liniowej

### Funkcja sigmoidalna (logistyczna)
$f(x) = \frac{1}{1 + e^{-x}}$ 


**zalety**
* gładki gradient funkcji
* przyjmuje wartości z zakresu 0-1 (normalizuje wyjście neuronu)
* przejrzysta predykcja (dla |X|>2 y przyjmuje wartości bliskie krawędzi funkcji logistycznej)

**wady**
* w przypadku bardzo wysokich/niskich wartości X, predykcja nie ulega zmianie; gradient zanika
* wyjścia nie są centrowane wokół 0
* mało wydajna obliczeniowo

### Funkcja Tanh (tangens hiperboliczny)
$f(x) = tan(h) = \frac{2}{1+e^{-2x}}-1$


**zalety**
* wyjścia centrowane są wokół 0 (ułatwia modelowanie danych, które są 'silnie' ujemne, 'silnie' dodatnie i neutralne

**wady**
* podobne jak w przypadku funkcji sigmoidalnej

### Funkcja ReLU (Rectified Linear Unit)
$f(x) \begin{cases} % changed and corrected
                    0 & jeśli \; x<0 \\
                    x &  jeśli \;x \geq 0.
                \end{cases}$


**zalety**
* wydajna obliczeniowo
* nieliniowość (ma pochodną!)

**wady**
* jeśli dane na wejściu są zerowe, lub ujemne sieć nie może przeprowadzić propagacji wstecznej

### Funkcja Leaky ReLU
$f(x) = max(0,1x, x)$


**zalety**
* nie ma problemu z danymi zerowymi, lub ujemnymi; niewielkie nachylenie w obszarze ujemym

**wady**
* niska dokładność predykcji dla wejść ujemnych

### Parametric ReLU
$f(x) = max(\alpha x, x)$


**zalety**
* nie przyjmuje stałej wartości 0.1, a jest dobierany odpowiedni współczynnik kierunkowy

**wady**
* dla każdego problemu może działać inaczej

### Softmax
$f(x) = \frac{exp(x_{i})}{\sum_{j}exp(x_{j}) }$


**zalety**
* możliwość klasyfikacji wielu klas
* przydatny dla warstwy wyjściowej

In [89]:
model.add(Dense(10, input_shape = (30,), activation = 'sigmoid'))
model.add(Dense(10, activation = 'sigmoid'))
model.add(Dense(10, activation = 'sigmoid'))
model.add(Dense(1, activation = 'sigmoid'))

Tworzymy optymalizator. Jego celem jest znalezienie minimum funkcji straty. Użyjemy tutaj optymalizatora SGD. Więcej optymizatorów dostępnych w keras można znaleźć [tutaj](https://keras.io/optimizers/).

In [90]:
sgd = optimizers.SGD(lr = 0.01)

Kompilujemy nasz model, podając funkcje optymalizującą, funkcję straty i miarę umożliwiającą kontrolowanie procesu uczenia. Definiujemy funkcję straty. Lista dostępnych funkcji straty znajduje się [tutaj](https://keras.io/losses/). Jedną z najbardziej znanych funkcji straty jest mean_squared_error, która używana jest również przy tworzeniu modelu regresji liniowej. Kolejnym argumentem jest metryka modelu. Zazwyczaj wybieramy dokładność (accuracy). Inne dostępne metryki znajdują się [tutaj](https://keras.io/metrics/). Inne, opcjonalne argumenty funkcji model.compile znajdują się  [tutaj](https://keras.io/models/model/#compile).

In [91]:
model.compile(optimizer = sgd, loss = 'binary_crossentropy', metrics = ['accuracy'])

Uczymy nasz model. Podajemy zbiór treningowy oraz jego etykiety. Definiujemy batch_size, epochs i verbose. 

In [92]:
model.fit(X_train, y_train, batch_size = 30, epochs = 100, verbose = 2)

Epoch 1/100
 - 0s - loss: 0.6881 - accuracy: 0.6396
Epoch 2/100
 - 0s - loss: 0.6779 - accuracy: 0.6396
Epoch 3/100
 - 0s - loss: 0.6723 - accuracy: 0.6396
Epoch 4/100
 - 0s - loss: 0.6685 - accuracy: 0.6396
Epoch 5/100
 - 0s - loss: 0.6663 - accuracy: 0.6396
Epoch 6/100
 - 0s - loss: 0.6634 - accuracy: 0.6396
Epoch 7/100
 - 0s - loss: 0.6615 - accuracy: 0.6396
Epoch 8/100
 - 0s - loss: 0.6604 - accuracy: 0.6396
Epoch 9/100
 - 0s - loss: 0.6591 - accuracy: 0.6396
Epoch 10/100
 - 0s - loss: 0.6584 - accuracy: 0.6396
Epoch 11/100
 - 0s - loss: 0.6578 - accuracy: 0.6396
Epoch 12/100
 - 0s - loss: 0.6569 - accuracy: 0.6396
Epoch 13/100
 - 0s - loss: 0.6562 - accuracy: 0.6396
Epoch 14/100
 - 0s - loss: 0.6557 - accuracy: 0.6396
Epoch 15/100
 - 0s - loss: 0.6555 - accuracy: 0.6396
Epoch 16/100
 - 0s - loss: 0.6552 - accuracy: 0.6396
Epoch 17/100
 - 0s - loss: 0.6552 - accuracy: 0.6396
Epoch 18/100
 - 0s - loss: 0.6550 - accuracy: 0.6396
Epoch 19/100
 - 0s - loss: 0.6550 - accuracy: 0.6396
Ep

In [93]:
results2 = model.evaluate(X_test, y_test)

200/200 [==============================] - 0s 704us/step


In [94]:
print('loss: ', results2[0])
print('accuracy: ', results2[1])

loss:  0.673589153289795
accuracy:  0.6050000190734863


**Loss (strata)** – to wynik dobrania wag modelu. Czym mniejsza wartości funkcji straty, tym lepiej. Strata pokazuje, jak złe były prognozy modelu. Jeśli predykcja jest idealna, strata wynosi 0. W przeciwieństwie do dokładności, strata nie jest mierzona w procentach. Jest to suma błędów popełnionych dla każdego przykładu w zestawach uczących lub walidacyjnych.\
**Accuracy (dokładność)** – miara dokładności prognozy modelu w stosunku do prawdziwych danych. Załóżmy, że masz 1000 próbek testowych i jeśli Twój model jest w stanie poprawnie sklasyfikować 990 z nich, dokładność modelu wyniesie 99,0%.

In [101]:
y_pred = model.predict(X_test)
y_pred_class = model.predict_classes(X_test)

## Miary oceny klasyfikatorów binarnych


#### Macierz błędu / tablica pomyłek (confusion matrix)

In [108]:
cm=confusion_matrix(y_test,y_pred_class)

In [109]:
print(cm)

[[  0  79]
 [  0 121]]


                                                        Macierz błędu

|                     | Stan faktyczny 0 | Stan faktyczny 1 |
|---------------------|----------------------|----------------------|
| Stan klasyfikacji 0 | TP                   | FP                   |
| Stan klasyfikacji 1 | FN                   | TN                   |

***Typy błędów:***
* TN - true negative, correct rejection - prawidłowa decyzja negatywna
* TP - true positive, hit - prawidłowa decyzja pozytywna
* FP - false positive, false alarm - błąd I rodzaju, wynik fałszywie pozytywny
* FN - false negative, with miss - błąd II rodzaju, wynik fałszywie negatywny

***Miary:***
* ***Czułość*** (sensitivity, true positive rate, TPR, recall) - Interpretacja: Prawdopodobieństwo, że klasyfikacja będzie poprawna, pod warunkiem, że przypadek jest pozytywyny. Prawdopodobieństwo, że klasyfikacja dla błędnego genotypu wykaże, że jest on błędny.

$TPR=\frac{TP}{P}=\frac{TP}{TP+FN}$

* ***Specyficzność*** (specificity (SPC), true negative rate, TNR) - Interpretacja: Prawdopodobieństwo, że klasyfikacja będzie poprawna, pod warunkiem, że przypadek jest negatywny. Prawdopodobieństwo, że klasyfikacja dla poprawnego genotypu wykaże, że jest on poprawny.

$SPC=\frac{TN}{N}=\frac{TN}{FP+TN}=1-FPR$

* ***Częstość fałszywych alaramów*** (false positive rate, FPR, fall-out) - Interpretacja: Jak duża frakcja genotypów poprawnych zostanie zakwalifikowana jako błędny?

$FPR=\frac{FP}{N}=\frac{FP}{FP+TN}=1-SPC$

* ***Częstość fałszywych odkryć*** (false discovery rate, FDR, fall-out) - Interpretacja: Jak duża frakcja spośród genotypów zakwalifikowanych jako błędne jest fałszywa?

$FDR=\frac{FP}{FP+TP}$

* ***Precyzja pozytywna*** (positive predictive value(PPV), precision) - Interpretacja: Jeśli genotyp został zaklasyfikowany jako błędny, jakie jest prawdopodobieństwwo, że genotyp jest rzeczywiście błędny?

$PPV=\frac{TP}{TP+FP}$

* ***Precyzja negatywna*** (negative predictive value(NPV)) - Interpretacja: Jeśli genotyp został zaklasyfikowany jako poprawny, jakie jest prawdopodobieństwwo, że genotyp jest rzeczywiście poprawny?

$PPV=\frac{TN}{TN+FN}$

* ***Dokładność*** (accuracy(ACC)) – Intepretacja: Prawdopodobieństwo prawidłowej klasyfikacji.

$ACC=\frac{TP+TN}{P+N}$

* ***F1-score*** – średnia harmoniczna z precyzji i czułości – ocena balansu między precyzją a czułością. Miara nie uwzględnia wyników prawdziwie negatywnych.

$F1=2\frac{PPV*TPR}{PPV+TPR}=\frac{2TP}{2TP+FP+FN}$

* ***$F_{\beta}$*** – uogólnienie powyższej miary. Parametr $\beta$ oznacza wagę jaką przykładamy do PPV.\
$F_{\beta}=(1+\beta^{2})\frac{PPV*TPR}{PPV\beta^{2}+TPR}$

* ***współczynnik korelacji Matthews (Matthews correlation coefficient)*** – zrównoważona miara oceny klasyfikatora. Uwzględnia wyniki prawdziwie i fałszywie pozytywne i negatywne. Może być stosowany, gdy klasy są różnej liczebności (niezbalansowane, unbalanced). MCC to współczynnik korelacji pomiędzy obserwowanymi i przewidywanymi klasyfikacjami binarnymi; przyjmuje wartości od -1 do 1, gdzie 1 to idealna klasyfikacja, 0 nie lepsza niż losowe przypisanie klasy, a -1 oznacza całkowitą rozbieżność pomiędzy klasyfikacją i stanem faktycznym.

$MCC=\frac{TP*TN-FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}$

Zadanie dodatkowe:
* Dowiedz się czym jest krzywa ROC oraz na jej podstawie wyznacz miarę AUC. 
* Dowiedz się czym jest kroswalidacja (sprawdzian krzyżowy, cross-validation) i spróbuj zastosować tę metodę do budowy klasyfikatora. 